# Experiment to observe "Intra-CCA fairness by Reno, Cubic and BBR at Edge and Core Scale"

## Set up your FABRIC environment


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

In [ ]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

## Get slice details

Put your slice name and the number of endpoints in the following cell:

In [ ]:
n_endpoints = 10
slice_name="bottleneck-" + str(n_endpoints) + '-test'

Then, load your slice details into the environment.slice = fablib.new_slice(name=slice_name)

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
sender_nodes = [slice.get_node(name='sender-' + str(i))  for i in range(n_endpoints)]
receiver_nodes = [slice.get_node(name='receiver-' + str(i))  for i in range(n_endpoints)]

In [ ]:
router_node = slice.get_node(name='router')
router_ingress_iface = router_node.get_interface(network_name = "link-sender")
router_egress_iface  = router_node.get_interface(network_name = "link-receiver")
router_egress_name = router_egress_iface.get_device_name()

## Generate flows

### Set experiment parameters

>delay, cca, test_duration, num_servers, flows

delay is the delay to be set at the receiver (20 ms,100 ms,200 ms)

num_servers is the number of ports to be opened on each receiver. For core scale we are opening 10 ports and for edge scale we are opening 1 port

test_duration is the time for which to send the iperf3 flows

cca1 is the first congestion control algorithm (bbr, reno or cubic); cca2 is the second congestion control algorithm (bbr, reno or cubic)

flows is the number of parallel flows to be send from each port

For sending 1000 flows set num_servers=10 and flows=10. This will send 100 flows from each of the 10 senders.

In [ ]:
cca1="bbr"
cca2="reno"
delay=20
test_duration=60
num_servers=10
flows=10

## Remove existing result files from the hosts

In [ ]:
for n in receiver_nodes:
    n.execute("rm -f ./*")

for n in sender_nodes:
    n.execute("rm -f ./*")

In [ ]:
for n in sender_nodes:
    n.execute("ls")

In [ ]:
for n in receiver_nodes:
    n.execute("ls")

### Set delay on the receiver

> Values: 20ms, 100ms, 200ms

Now set up delay on the receiver interface:

First delete any existing queue (don't worry if there is an error, it means there was not!)

In [ ]:
for n in receiver_nodes:
    receiver_inf=n.get_interface(network_name= "link-receiver")
    receiver_inf_name = receiver_inf.get_device_name()
    n.execute("sudo tc qdisc del dev " + receiver_inf_name + " root netem")
    n.execute("sudo tc qdisc add dev " + receiver_inf_name + " root netem delay " + str(delay) +"ms limit 1000000")
    

### Start parallel servers on the receivers

In this, the base_port is the starting address of port number

Now start the `iperf3` flows:

In [ ]:

        
base_port=50000
for i in range(0,10):
    receiver_nodes[i].execute("sudo killall iperf3")
for i in range(0,10):
    for k in range (1,num_servers+1): 
        
        server_port=base_port+k
        report_file=str(server_port)+"-server.dat"
        receiver_nodes[i].execute_thread("iperf3 -s -p "+str(server_port)+" -D --logfile "+report_file)

In [ ]:
#check files in all receiver
for n in receiver_nodes:
    n.execute("ls")

### Start parallel clients on the senders

In this, the base_port is the starting address of port number on the receiver


In [ ]:
import time
base_port=60000


for n in sender_nodes:
    n.execute("sudo killall iperf3")

for i,n in enumerate(sender_nodes):
    if i<5:
        for k in range (0,num_servers):
            server_port=base_port+k+1
            report_file="sender-10.10.2.1"+str(k)+"-"+str(server_port)+"-"+str(test_duration)+"-"+cca1+".txt"
            n.execute_thread("iperf3 -c 10.10.2.1"+str(i)+ "-p "+str(server_port)+ " -t "+str(test_duration)+ " -C "+cca1+" -P "+str(flows)+ " --format k $>"+report_file+ " &")
    else:
        for k in range (0,num_servers):
            server_port=base_port+k+1
            report_file="sender-10.10.2.1"+str(k)+"-"+str(server_port)+"-"+str(test_duration)+"-"+cca2+".txt"
            n.execute_thread("iperf3 -c 10.10.2.1"+str(i)+ "-p "+str(server_port)+ " -t "+str(test_duration)+ " -C "+cca2+" -P "+str(flows)+ " --format k $>"+report_file+ " &")
time.sleep(test_duration+300)              

In [ ]:
#check files on all the senders
for n in sender_nodes:
    n.execute("ls")

## Analyze the results

Transfer files from hosts to router

Calculate sum of bandwidth, square of sum of bandwidth, count of flows and jfi:

In [ ]:


# for n in sender_nodes:
#     (sum_sen, serr)=n.execute("grep -r -E \"[0-9].*0.00-"+str(test_duration)+".*sender\" . |awk '{sum+=$7}END {print sum}'")
#     sum_bw.append(float(sum_sen.strip()))
#     (sum_sq, sqerr)=n.execute("grep -r -E \"[0-9].*0.00-"+str(test_duration)+".*sender\" . |awk '{sum+=$7*$7}END {print sum}'")
#     sum_sq_bw.append(float(sum_sq.strip()))
#     (ncount, ncerr)=n.execute("grep -r -E \"[0-9].*0.00-"+str(test_duration)+".*sender\" . |awk '{count+=1}END {print count}'")
#     count_flow.append(int(ncount.strip()))



sum_bw1 = []
count_flow1 = []

sum_bw2 = []
count_flow2 = []

for i,n in enumerate(sender_nodes):
    if i<5:
        (sum_sen1, serr1)=n.execute("grep -r -E \"[0-9].*0.00-[0-9].*sender --include *"+cca1+".txt . |awk '{sum+=$7}END {print sum}'")
        sum_bw1.append(float(sum_sen.strip()))
        (ncount1,ncerr1)=n.execute("grep -r -E \"[0-9].*0.00-[0-9].*sender\" --include *"+cca1+".txt . |awk '{count+=1}END {print count}'")
        count_flow1.append(ncount1.strip())
    else:
        (sum_sen2, serr2)=n.execute("grep -r -E \"[0-9].*0.00-[0-9].*sender\" --include *"+cca2+".txt . |awk '{sum+=$7}END {print sum}'")
        sum_bw2.append(float(sum_sen2.strip()))
        (ncount2,ncerr2)=n.execute("grep -r -E \"[0-9].*0.00-[0-9].*sender\"--include *"+cca2+".txt . |awk '{count+=1}END {print count}'")
        count_flow2.append(ncount2.strip())


tput1=sum(sum_bw1)
c1=sum(count_flow1)

print("Sum of bandwidth of "+cca1+ " is %f Kbits/sec " % tput1)
print("Count of flows of " +cca1+ " is " + c1)

tput2=sum(sum_bw2)
c2=sum(count_flow2)

print("Sum of bandwidth of "+cca2+ " is %f Kbits/sec " % tput2)
print("Count of flows of " +cca2+ " is " + c2)

In [ ]:
import csv
import sys
import os

jfi_filename='jfi.csv'
if not os.path.isfile(jfi_filename):
    with open(jfi_filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        header ='CCA1', 'CCA2', 'Duration of Expt(sec)', 'Base RTT(ms)', 'Total Bandwidth(Kbps)', 'BW_CCA1', 'BW_CCA2', 'Count_CCA1', 'Count_CCA2', 'BW_CCA1/BW'
        writer.writerow(header)
    
with open(jfi_filename, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    columns = cca1, cca2, test_duration, delay, tput1+tput2, tput1, tput2, c1, c2, tput2/(tput1+tput2)
    writer.writerow(columns)
